## In PMI cm03
### Environment
```bash
source activate scanpy_1.9.3
ipython --profile=ak1
```

In [ ]:
from anndata import AnnData
import anndata
from scipy import sparse, io
import scipy
import pandas as pd
import scipy.io
import os
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors
from matplotlib.ticker import StrMethodFormatter
matplotlib.use('TkAgg')
import numpy as np
import seaborn as sns
import math
import scanpy.external as sce
#import scrublet as scr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import zscore
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.stats.multitest import multipletests
sns.set(font="arial", font_scale=1.15, style='ticks')
plt.rcParams['figure.figsize'] = (6,6)
plt.rc("axes.spines", top=False, right=False)
sc.settings.verbosity = 3

%autoindent
%matplotlib

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#104e8b", "#ffdab9", "#8b0a50"])
sample_palette = {'AK1':'#FF0000', 'iPSC':'#154360','NPC':'#229954'}


# Initial Diagnosis

In [ ]:
ak1 = pd.read_table("AK1_collapsed_classification.filtered_lite_classification_saturation.txt")
ak1_melt = pd.melt(ak1, id_vars=['reads'], value_vars=['unique_genes', 'unique_isoforms', 'unique_genes_known', 'unique_isoforms_known'])
sns.lineplot(data=ak1_melt, x='reads', y='value', hue='variable')
sns.plt.savefig('lineplot.pdf')

ipsc = pd.read_table("iPSC_collapsed_classification.filtered_lite_classification_saturation.txt")
ipsc_melt = pd.melt(ipsc, id_vars=['reads'], value_vars=['unique_genes', 'unique_isoforms', 'unique_genes_known', 'unique_isoforms_known'])
sns.lineplot(data=ipsc_melt, x='reads', y='value', hue='variable')

npc = pd.read_table("NPC_collapsed_classification.filtered_lite_classification_saturation.txt")
npc_melt = pd.melt(npc, id_vars=['reads'], value_vars=['unique_genes', 'unique_isoforms', 'unique_genes_known', 'unique_isoforms_known'])
sns.lineplot(data=npc_melt, x='reads', y='value', hue='variable')


# Single-Cell Analysis

In [ ]:
ak1 = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/AK1/genes_seurat', cache=True)
ipsc = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/iPSC/genes_seurat', cache=True)
npc = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/NPC/genes_seurat', cache=True)

ak1_iso = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/AK1/isoforms_seurat', cache=True)
ipsc_iso = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/iPSC/isoforms_seurat', cache=True)
npc_iso = sc.read_10x_mtx('/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/NPC/isoforms_seurat', cache=True)

ak1.write(filename="AK1_genes.h5ad")
ipsc.write(filename="iPSC_genes.h5ad")
npc.write(filename="NPC_genes.h5ad")
ak1_iso.write(filename="AK1_isoforms.h5ad")
ipsc_iso.write(filename="iPSC_isoforms.h5ad")
npc_iso.write(filename="NPC_isoforms.h5ad")

'''
knee = np.sort((np.array(ak1.X.sum(axis=1))).flatten())[::-1] # UMI count for each cell (axis=1)
cell_set = np.arange(len(knee))
cutoff = 200
num_cells = cell_set[knee > cutoff][::-1][0]

fig, ax = plt.subplots(figsize=(10, 7))

ax.loglog(knee, cell_set, lw=5, color="g")
ax.axvline(x=num_cells, lw=3, color="k")
ax.axhline(y=cutoff, lw=3, color="k")
ax.set_xlabel("Set of Barcodes")
ax.set_ylabel("UMI counts")
ax.grid(True, which = "both")

print(f"{num_cells:,.0f} cells passed the {cutoff} UMI threshold")
#9,364 cells passed the 200 UMI threshold (2022-10-22)
'''

ak1 = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/AK1/genes_seurat', cache=True)
ipsc = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/iPSC/genes_seurat', cache=True)
npc = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/NPC/genes_seurat', cache=True)
ak1.var_names_make_unique()
ipsc.var_names_make_unique()
npc.var_names_make_unique()

ak1_iso = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/AK1/isoforms_seurat', cache=True)
ipsc_iso = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/iPSC/isoforms_seurat', cache=True)
npc_iso = sc.read_10x_mtx('/mnt/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/New_2024/NEW/NPC/isoforms_seurat', cache=True)
ak1_iso.var_names_make_unique()
ipsc_iso.var_names_make_unique()
npc_iso.var_names_make_unique()


In [ ]:
ak1.var['mt'] = ak1.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(ak1, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
ipsc.var['mt'] = ipsc.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(ipsc, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
npc.var['mt'] = npc.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(npc, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

ak1_iso.var['mt'] = ak1_iso.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(ak1_iso, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
ipsc_iso.var['mt'] = ipsc_iso.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(ipsc_iso, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
npc_iso.var['mt'] = npc_iso.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(npc_iso, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
# Gene-level
integrated = AnnData.concatenate(ak1, ipsc, npc, join='outer', batch_categories = ['AK1', 'iPSC', 'NPC'], index_unique = '-')
integrated.raw = integrated
integrated.var['mt'] = integrated.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(integrated, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)
sc.pp.log1p(integrated)
#integrated.write_h5ad("integrated.h5ad")

# Isoform-level
integrated_iso = AnnData.concatenate(ak1_iso, ipsc_iso, npc_iso, join='outer', batch_categories = ['AK1', 'iPSC', 'NPC'], index_unique = '-')
integrated_iso.raw = integrated_iso
integrated_iso.var['mt'] = integrated_iso.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(integrated_iso, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)
sc.pp.log1p(integrated_iso)
#integrated_iso.write_h5ad("integrated_iso.h5ad")

In [ ]:
# Wilcoxon
## Gene-level
sc.tl.rank_genes_groups(integrated, groupby='batch', method='wilcoxon', corr_method='benjamini-hochberg', use_raw=False, pts=True, key_added='DEG_bw_batches_wilcoxon') # key_added=''
dedf_ak1 = sc.get.rank_genes_groups_df(integrated, group='AK1', key='DEG_bw_batches_wilcoxon')
dedf_ipsc = sc.get.rank_genes_groups_df(integrated, group='iPSC', key='DEG_bw_batches_wilcoxon')
dedf_npc = sc.get.rank_genes_groups_df(integrated, group='NPC', key='DEG_bw_batches_wilcoxon')

#dedf_ak1[(dedf_ak1['logfoldchanges'] > 2) & (dedf_ak1['pvals_adj'] < 10e-5)].sort_values(by='logfoldchanges', ascending=False)

ak1_up_log2fc_2_genename = set(dedf_ak1[(dedf_ak1['logfoldchanges'] > 2) & (dedf_ak1['pvals_adj'] < 10e-5)]['names'].values)
ak1_up_log2fc_1_genename = set(dedf_ak1[(dedf_ak1['logfoldchanges'] > 1) & (dedf_ak1['pvals_adj'] < 10e-5)]['names'].values)
ak1_down_log2fc_2_genename = set(dedf_ak1[(dedf_ak1['logfoldchanges'] < -2) & (dedf_ak1['pvals_adj'] < 10e-5)]['names'].values)
ak1_down_log2fc_1_genename = set(dedf_ak1[(dedf_ak1['logfoldchanges'] < -1) & (dedf_ak1['pvals_adj'] < 10e-5)]['names'].values)

ak1_deg_log2fc_1_genename = ak1_up_log2fc_1_genename.union(ak1_down_log2fc_1_genename)
total_genename = set(integrated.var.index)
ak1_nondeg_log2fc_1_genename = total_genename - ak1_deg_log2fc_1_genename

## Isoform-level
sc.tl.rank_genes_groups(integrated_iso, groupby='batch', method='wilcoxon', corr_method='benjamini-hochberg', use_raw=False, pts=True, key_added='DEG_bw_batches_wilcoxon') # key_added=''
dedf_ak1_iso = sc.get.rank_genes_groups_df(integrated_iso, group='AK1', key='DEG_bw_batches_wilcoxon')
dedf_ipsc_iso = sc.get.rank_genes_groups_df(integrated_iso, group='iPSC', key='DEG_bw_batches_wilcoxon')
dedf_npc_iso = sc.get.rank_genes_groups_df(integrated_iso, group='NPC', key='DEG_bw_batches_wilcoxon')

#dedf_ak1_iso[(dedf_ak1_iso['logfoldchanges'] > 2) & (dedf_ak1_iso['pvals_adj'] < 10e-5)].sort_values(by='logfoldchanges', ascending=False)

ak1_iso_up_log2fc_2_transname = set(dedf_ak1_iso[(dedf_ak1_iso['logfoldchanges'] > 2) & (dedf_ak1_iso['pvals_adj'] < 10e-5)]['names'].values)
ak1_iso_up_log2fc_1_transname = set(dedf_ak1_iso[(dedf_ak1_iso['logfoldchanges'] > 1) & (dedf_ak1_iso['pvals_adj'] < 10e-5)]['names'].values)
ak1_iso_down_log2fc_2_transname = set(dedf_ak1_iso[(dedf_ak1_iso['logfoldchanges'] < -2) & (dedf_ak1_iso['pvals_adj'] < 10e-5)]['names'].values)
ak1_iso_down_log2fc_1_transname = set(dedf_ak1_iso[(dedf_ak1_iso['logfoldchanges'] < -1) & (dedf_ak1_iso['pvals_adj'] < 10e-5)]['names'].values)

ak1_iso_dex_log2fc_1_transname = ak1_iso_up_log2fc_1_transname | ak1_iso_down_log2fc_1_transname

ak1_iso_up_log2fc_1_genename = set(map(lambda x: '-'.join(x.split('-')[:-1]), ak1_iso_up_log2fc_1_transname))
ak1_iso_down_log2fc_1_genename = set(map(lambda x: '-'.join(x.split('-')[:-1]), ak1_iso_down_log2fc_1_transname))
ak1_iso_dex_log2fc_1_genename = ak1_iso_up_log2fc_1_genename | ak1_iso_down_log2fc_1_genename

ak1_iso_dex_log2fc_2_transname = ak1_iso_up_log2fc_2_transname | ak1_iso_down_log2fc_2_transname
ak1_iso_up_log2fc_2_genename = set(map(lambda x: '-'.join(x.split('-')[:-1]), ak1_iso_up_log2fc_2_transname))
ak1_iso_down_log2fc_2_genename = set(map(lambda x: '-'.join(x.split('-')[:-1]), ak1_iso_down_log2fc_2_transname))
ak1_iso_dex_log2fc_2_genename = ak1_iso_up_log2fc_2_genename | ak1_iso_down_log2fc_2_genename

# Intersection of non-deg (Gene-level) and deg (Isoform-level)
ak1_nodeg_log2fc_1_yesdex_log2fc_1_genename = ak1_nondeg_log2fc_1_genename & ak1_iso_dex_log2fc_1_genename #len(ak1_nodeg_yesdex_log2fc_1_genename) == 1365
ak1_nodeg_log2fc_1_yesdex_log2fc_2_genename = ak1_nondeg_log2fc_1_genename & ak1_iso_dex_log2fc_2_genename #len(ak1_nodeg_yesdex_log2fc_1_genename) == 1365

ak1_nodeg_log2fc_1_yesdex_log2fc_2_transname = set()
for iso in dedf_ak1_iso['names'].values:
    if '-'.join(iso.split('-')[:-1]) in ak1_nodeg_log2fc_1_yesdex_log2fc_2_genename:
        ak1_nodeg_log2fc_1_yesdex_log2fc_2_transname.add(iso)

dedf_ak1_iso[dedf_ak1_iso['names'].isin(ak1_nodeg_log2fc_1_yesdex_log2fc_2_transname)]

In [ ]:
# EdgeR
## Aggregate single cells into pseudo-replicates (https://www.sc-best-practices.org/conditions/differential_gene_expression.html#)
NUM_OF_CELL_PER_DONOR = 30


def aggregate_and_filter(
    adata,
    cell_identity,
    donor_key="sample",
    condition_key="label",
    cell_identity_key="cell_type",
    obs_to_keep=[],  # which additional metadata to keep, e.g. gender, age, etc.
    replicates_per_sample=3,
):
    # subset adata to the given cell identity
    adata_cell_pop = adata[adata.obs[cell_identity_key] == cell_identity].copy()
    # check which donors to keep according to the number of cells specified with NUM_OF_CELL_PER_DONOR
    size_by_donor = adata_cell_pop.obs.groupby([donor_key]).size()
    donors_to_drop = [
        donor
        for donor in size_by_donor.index
        if size_by_donor[donor] <= NUM_OF_CELL_PER_DONOR
    ]
    if len(donors_to_drop) > 0:
        print("Dropping the following samples:")
        print(donors_to_drop)
    df = pd.DataFrame(columns=[*adata_cell_pop.var_names, *obs_to_keep])

    adata_cell_pop.obs[donor_key] = adata_cell_pop.obs[donor_key].astype("category")
    for i, donor in enumerate(donors := adata_cell_pop.obs[donor_key].cat.categories):
        print(f"\tProcessing donor {i+1} out of {len(donors)}...", end="\r")
        if donor not in donors_to_drop:
            adata_donor = adata_cell_pop[adata_cell_pop.obs[donor_key] == donor]
            # create replicates for each donor
            indices = list(adata_donor.obs_names)
            random.shuffle(indices)
            indices = np.array_split(np.array(indices), replicates_per_sample)
            for i, rep_idx in enumerate(indices):
                adata_replicate = adata_donor[rep_idx]
                # specify how to aggregate: sum gene expression for each gene for each donor and also keep the condition information
                agg_dict = {gene: "sum" for gene in adata_replicate.var_names}
                for obs in obs_to_keep:
                    agg_dict[obs] = "first"
                # create a df with all genes, donor and condition info
                df_donor = pd.DataFrame(adata_replicate.X.A)
                df_donor.index = adata_replicate.obs_names
                df_donor.columns = adata_replicate.var_names
                df_donor = df_donor.join(adata_replicate.obs[obs_to_keep])
                # aggregate
                df_donor = df_donor.groupby(donor_key).agg(agg_dict)
                df_donor[donor_key] = donor
                df.loc[f"donor_{donor}_{i}"] = df_donor.loc[donor]
    print("\n")
    # create AnnData object from the df
    adata_cell_pop = sc.AnnData(
        df[adata_cell_pop.var_names], obs=df.drop(columns=adata_cell_pop.var_names)
    )
    return adata_cell_pop

In [ ]:
'''
sce <- zellkonverter::readH5AD("integrated.h5ad", verbose=TRUE)
#✔ Read ./integrated.h5ad [252ms]
#ℹ uns is empty and was skipped
#✔ X matrix converted to assay [5.7s]
#ℹ layers is empty and was skipped
#! The names of these selected var columns have been modified to match R conventions: 'gene_ids-AK1' -> 'gene_ids.AK1', 'gene_ids-NPC' ->
#'gene_ids.NPC', and 'gene_ids-iPSC' -> 'gene_ids.iPSC'
#✔ var converted to rowData [183ms]
#✔ obs converted to colData [48ms]
#ℹ varm is empty and was skipped
#ℹ obsm is empty and was skipped
#ℹ varp is empty and was skipped
#ℹ obsp is empty and was skipped
#✔ SingleCellExperiment constructed [390ms]
#ℹ Skipping conversion of raw
#✔ Converting AnnData to SingleCellExperiment ... done
#Warning message:
#The names of these selected var columns have been modified to match R conventions: 'gene_ids-AK1' -> 'gene_ids.AK1', 'gene_ids-NPC' ->
#'gene_ids.NPC', and 'gene_ids-iPSC' -> 'gene_ids.iPSC' 

sce2seu <- as.Seurat(sce, counts="X", data=NULL)

'''

In [ ]:
sc.pl.violin(integrated, ['total_counts', 'n_genes_by_counts'], groupby='batch', size=2, log=True, cut=0, inner='quartile', ylabel=['UMI counts', 'Gene counts'], palette=sample_palette, rotation=0.1)
sc.pl.violin(integrated, ['pct_counts_mt'], groupby='batch', size=2, log=False, cut=0, inner='quartile', ylabel=['Percentage of mitochondrial genes'], palette=sample_palette, rotation=0.1)

## Analysis (Treat them like Bulk-cell Iso-Seq)

In [ ]:
counts_all = integrated.var[['gene_ids-AK1', 'gene_ids-iPSC', 'gene_ids-NPC', 'total_counts-AK1', 'total_counts-iPSC', 'total_counts-NPC']].copy()
#counts_all.to_csv("Merged_counts.tab", sep='\t', index=True)

In [ ]:
counts_all_iso = integrated_iso.var[['gene_ids-AK1', 'gene_ids-iPSC', 'gene_ids-NPC', 'total_counts-AK1', 'total_counts-iPSC', 'total_counts-NPC']].copy()
#counts_all_iso.to_csv("Merged_counts_iso.tab", sep='\t', index=True)

In [ ]:
def cpm_calculator(pdseries):
    scale_factor = pdseries.sum()
    return pdseries * 10**6 / scale_factor

#counts_all = pd.read_table("Merged_counts.tab", index_col=0)
for sampleid in ['AK1', 'iPSC', 'NPC']:
    counts_all[f'CPM-{sampleid}'] = cpm_calculator(counts_all[f'total_counts-{sampleid}'])


#counts_all_iso = pd.read_table("Merged_counts_iso.tab", index_col=0)
for sampleid in ['AK1', 'iPSC', 'NPC']:
    counts_all_iso[f'CPM-{sampleid}'] = cpm_calculator(counts_all_iso[f'total_counts-{sampleid}'])
    

In [ ]:
# Unfiltered
fig, ax = plt.subplots(1, 1, figsize=(5, 5), constrained_layout=True)
sns.violinplot(np.log1p(counts_all.iloc[:, -3:]), palette={'CPM-AK1':'#FF0000', 'CPM-iPSC':'#154360','CPM-NPC':'#229954'}, ax=ax)
ax.set_xticklabels(['AK1', 'iPSC', 'NPC'])
ax.set_ylabel('log[CPM+1]')

In [ ]:
cpm_cutoff = 2
counts_all_filtered = counts_all[counts_all.iloc[:, -3:].min(axis=1) >= cpm_cutoff]
cpm_all_filtered = counts_all_filtered.iloc[:, -3:].copy()
cpm_all_filtered_zero = cpm_all_filtered.fillna(0)
cpm_all_filtered_zero_log1p = np.log1p(cpm_all_filtered_zero)
cpm_all_filtered_zero_log1p_zscore = cpm_all_filtered_zero_log1p.apply(zscore, axis=1)

cpm_cutoff = 2
counts_all_iso_filtered = counts_all_iso[counts_all_iso.iloc[:, -3:].min(axis=1) >= cpm_cutoff]
cpm_all_iso_filtered = counts_all_iso_filtered.iloc[:, -3:].copy()
cpm_all_iso_filtered_zero = cpm_all_iso_filtered.fillna(0)
cpm_all_iso_filtered_zero_log1p = np.log1p(cpm_all_iso_filtered_zero)
cpm_all_iso_filtered_zero_log1p_zscore = cpm_all_iso_filtered_zero_log1p.apply(zscore, axis=1)

In [ ]:
# Maybe centering is needed here
clustergrid = sns.clustermap(cpm_all_filtered_zero_log1p_zscore, method='ward', metric='euclidean', cmap=cmap)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])

clustergrid = sns.clustermap(cpm_all_iso_filtered_zero_log1p_zscore, method='ward', metric='euclidean', cmap=cmap)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])



# No DEG (|log2FC|<1) Yes DEX (|log2FC|>2)
# Isoforms (zscore)
clustergrid = sns.clustermap(cpm_all_iso_filtered_zero_log1p_zscore[cpm_all_iso_filtered_zero_log1p_zscore.index.isin(ak1_nodeg_log2fc_1_yesdex_log2fc_2_transname)], method='ward', metric='euclidean', cmap='viridis', xticklabels=True, yticklabels=False)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])
# Isoforms (just log1p)
clustergrid = sns.clustermap(cpm_all_iso_filtered_zero_log1p[cpm_all_iso_filtered_zero_log1p.index.isin(ak1_nodeg_log2fc_1_yesdex_log2fc_2_transname)], method='ward', metric='euclidean', cmap='viridis', xticklabels=True, yticklabels=False)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])
# Genes (zscore)=> it lloks like they are DEG...
clustergrid = sns.clustermap(cpm_all_filtered_zero_log1p_zscore[cpm_all_filtered_zero_log1p_zscore.index.isin(ak1_nodeg_log2fc_1_yesdex_log2fc_2_genename)], method='ward', metric='euclidean', cmap='viridis', xticklabels=True, yticklabels=False)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])
# Genes (just log1p)=> it lloks like they are DEG...
clustergrid = sns.clustermap(cpm_all_filtered_zero_log1p[cpm_all_filtered_zero_log1p.index.isin(ak1_nodeg_log2fc_1_yesdex_log2fc_2_genename)], method='ward', metric='euclidean', cmap='viridis', xticklabels=True, yticklabels=False)
clustergrid.ax_heatmap.set_xticklabels(['AK1', 'iPSC', 'NPC'])

##### First, execute *_Run_DropletUtils.R*

## Preprocessing on filtered CBC-UMI matrix

#### Doublet detection (using scrublet version 0.2.3) and removal for each sample

In [ ]:
for sample in [ak1, ipsc, npc]:
    sce.pp.scrublet(sample, adata_sim=None, sim_doublet_ratio=2.0, expected_doublet_rate=0.08, stdev_doublet_rate=0.02, synthetic_doublet_umi_subsampling=1.0, knn_dist_metric='euclidean', n_prin_comps=30, verbose=True)

'''
# Filtering
ak1 = ak1[ak1.obs['predicted_doublet'] == False, :]
ipsc = ipsc[ipsc.obs['predicted_doublet'] == False, :]
npc = npc[npc.obs['predicted_doublet'] == False, :]

for sample in [ak1, ipsc, npc]:
    sample.obs['n_counts'] = sample.X.sum(axis=1)
    sample.obs['n_genes'] = (sample.X > 0).sum(axis=1)

q20 = ak1.obs['n_counts'].quantile(q=0.20, interpolation='linear') # 20% quantile value
ak1 = ak1[ak1.obs['n_counts'] > q20, :]

q20 = ipsc.obs['n_counts'].quantile(q=0.20, interpolation='linear') # 20% quantile value
ipsc = ipsc[ipsc.obs['n_counts'] > q20, :]

q20 = npc.obs['n_counts'].quantile(q=0.20, interpolation='linear') # 20% quantile value
npc = npc[npc.obs['n_counts'] > q20, :]
'''

#### Batch integration

In [ ]:
integrated = AnnData.concatenate(ak1, ipsc, npc, join='outer', batch_categories = ['AK1', 'iPSC', 'NPC'], index_unique = '-')
integrated.raw = integrated
integrated.var['mt'] = integrated.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(integrated, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

integrated.var[integrated.var.index.isin(integrated.var_names[integrated.var_names.str.contains('\+')])]
#Checked Distribution of UMI counts and Gene counts
#integrated.obs['n_counts'] = integrated.X.sum(1)
#integrated.obs['n_genes'] = (integrated.X > 0).sum(1)
#sns.set(font="Arial", font_scale=1.5, style='ticks')
#sc.pl.violin(integrated, ['n_counts', 'n_genes'], groupby='batch', size=2, log=True, cut=0, inner='quartile', ylabel=['UMI counts', 'Gene counts'], rotation=0.1)
#sns.despine()
#sns.set(font="Arial", font_scale=1, style='ticks') # Back to original settings

# For making background gene-set for performing GSEA
with open("Background_GeneSets_from_MAS-ISO-Seq.txt", 'w') as rfh:
    for i in integrated.var.index:
        rfh.write(i + '\n')
        rfh.flush()

sc.pl.violin(integrated, ['total_counts', 'n_genes_by_counts'], groupby='batch', size=2, log=True, cut=0, inner='quartile', ylabel=['UMI counts', 'Gene counts'], palette=sample_palette, rotation=0.1)
sc.pl.violin(integrated, ['pct_counts_mt'], groupby='batch', size=2, log=False, cut=0, inner='quartile', ylabel=['Percentage of mitochondrial genes'], palette=sample_palette, rotation=0.1)

cpm_counts = sc.pp.normalize_total(integrated, target_sum=1e6, inplace=False)


In [ ]:
sc.pp.filter_genes(integrated, min_cells=0) # 'n_cells' added in integrated.var 
sc.pp.filter_genes(integrated, min_cells=5) # 'n_cells' added in integrated.var 

cpm_counts = sc.pp.normalize_total(integrated, target_sum=1e6, inplace=False)

'''
In [142]: cpm_counts
Out[142]: 
{'X': <24634x45853 sparse matrix of type '<class 'numpy.float32'>'
 	with 17515606 stored elements in Compressed Sparse Row format>,
 'norm_factor': array([2330., 1335., 3835., ...,  200.,  283.,  204.], dtype=float32)}
 
cpm_counts['norm_factor'] == integrated.obs['total_counts'] # CPM = counts per million (gene count*10^6/total counts)
'''
integrated.layers["cpm"] = cpm_counts["X"]
integrated.layers["log1p_cpm"] = sc.pp.log1p(cpm_counts["X"], copy=True)

fig, axes = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
p1 = sns.histplot(integrated.obs["total_counts"], bins=100, kde=True, ax=axes[0])
axes[0].set_title("Total counts")
axes[0].set_xlabel('Total counts in each cell')
axes[0].xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
p2 = sns.histplot(integrated.layers["log1p_cpm"].sum(1), bins=100, kde=True, ax=axes[1])
axes[1].set_title("Shifted logarithm (CPM)")
axes[1].set_xlabel('Total log(CPM+1) in each cell')
axes[1].xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[1].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[1].legend().remove()
'''p3 = sns.histplot(integrated.layers["cpm"].sum(1), bins=100, kde=True, ax=axes[2])
axes[2].set_title("CPM")
axes[2].legend().remove()
This would just be collection of 10^6
'''

# By Sample
fig, axes = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)
p1 = sns.histplot(integrated[integrated.obs['batch'] == 'AK1'].layers["log1p_cpm"].sum(1), bins=100, kde=True, ax=axes[0])
axes[0].set_title("AK1")
axes[0].set_xlabel('Total log(CPM+1) in each cell')
axes[0].xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[0].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[0].legend().remove()
p2 = sns.histplot(integrated[integrated.obs['batch'] == 'iPSC'].layers["log1p_cpm"].sum(1), bins=100, kde=True, ax=axes[1])
axes[1].set_title("iPSC")
axes[1].set_xlabel('Total log(CPM+1) in each cell')
axes[1].xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[1].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[1].legend().remove()
p3 = sns.histplot(integrated[integrated.obs['batch'] == 'NPC'].layers["log1p_cpm"].sum(1), bins=100, kde=True, ax=axes[2])
axes[2].set_title("NPC")
axes[2].set_xlabel('Total log(CPM+1) in each cell')
axes[2].xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[2].yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
axes[2].legend().remove()
#integrated.write(filename='/data/Projects/phenomata/01.Projects/13.AK1_PacBio/02.RNA/integrated_20230304.h5ad')

In [ ]:
integrated.X = integrated.layers['log1p_norm'].copy()
sc.pp.highly_variable_genes(integrated)
integrated.var['highly_variable'].value_counts() # 3,494 ==> 2023-10-13

integrated.raw = integrated
#sc.pp.scale(integrated, max_value=10) # tabula muris senis default (2021-08-10) # mean and std on adata.var
#sc.pp.scale(test3, zero_center=True, max_value=10, copy=False, layer=None, obsm=None)

np.random.seed(0)
rand_is = np.random.permutation(list(range(integrated.shape[0])))

sc.tl.pca(integrated, n_comps=100, use_highly_variable=True, svd_solver='arpack')
sc.pl.pca(integrated, color=['batch', 'total_counts'], legend_loc='on data', size=15, add_outline=False, color_map='CMRmap', palette=sample_palette, components=['1,2'], title=['', 'UMI count'], annotate_var_explained=True)
sc.pl.pca(integrated, color=['batch', 'total_counts'], legend_loc='on data', size=15, add_outline=False, color_map='CMRmap', palette=sample_palette, components=['1,3'], title=['', 'UMI count'], annotate_var_explained=True)
sc.pl.pca(integrated, color=['batch', 'total_counts'], legend_loc='on data', size=15, add_outline=False, color_map='CMRmap', palette=sample_palette, components=['1,4'], title=['', 'UMI count'], annotate_var_explained=True)
sc.pl.pca_variance_ratio(integrated, n_pcs=100, log=True)
sc.pl.pca_loadings(integrated, components='1,2,3,4', include_lowest=True)

cell_cycle_genes=[x.strip()[0] + x.strip()[1:] for x in open("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/Database/regev_lab_cell_cycle_genes.txt")]
s_genes= cell_cycle_genes[:43]
g2m_genes= cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in integrated.var_names]
sc.tl.score_genes_cell_cycle(integrated, s_genes=s_genes, g2m_genes=g2m_genes)
"""
Used 'raw' attribute of adata (use_raw = True if .raw is present)
So, log-tranformed scran-normalized counts are put into score_genes_cell_cycle function
"""

df = integrated.obs[['batch', 'phase']]
ax = pd.crosstab(df['batch'], df['phase'], normalize='index', margins=True).plot.bar(stacked=True, rot=45, color={'S': '#689aff', 'G2M': '#fdbf6f', 'G1': '#b15928'})
ax.legend(loc='upper left', bbox_to_anchor=(1.01, 0.25), frameon=False)
ax.set_ylabel('Proportion of Cell Cycle Phase')
ax.set_xlabel("")
plt.tight_layout()
sns.despine()

sc.pp.neighbors(integrated, n_neighbors=15, n_pcs=10)
sc.tl.umap(integrated)
sc.pl.umap(integrated, color=['batch', 'total_counts'], legend_loc='on data', size=15, add_outline=False, color_map='CMRmap', palette=sample_palette, components=['1,2'], title=['', 'UMI count'])

sc.tl.draw_graph(integrated, layout='fa', init_pos=None, neighbors_key=None) ## init_pos가 .obsm에 있는 pca, umap, paga 등이 될 수 있다.
sc.pl.draw_graph(integrated, color=['batch', 'total_counts'], add_outline=True, legend_loc='on data', size=15, color_map='CMRmap')

'''
#sce.pp.bbknn default ==> n_pcs=50, neighbors_within_batch=3, trim=None, annoy_n_trees=10,
sce.pp.bbknn(integrated, batch_key='batch', n_pcs=20, neighbors_within_batch=5, trim=None)
sc.tl.umap(integrated, min_dist=0.5, spread=1.0, n_components=2, alpha=1.0, gamma=1.0, init_pos='spectral', method='umap')
#integrated.uns['batch_colors'] = ['#689aff', '#fdbf6f', '#b15928']
sc.pl.umap(integrated, color=['batch'], add_outline=False, legend_loc='on data', size=20, title='')
sns.despine()
'''
fig, axes = plt.subplots(3,1,figsize=(4.5,15.5))
for i in range(len(integrated.obs['batch'].cat.categories)):
    sc.pl.umap(integrated, color=['batch'], add_outline=False, legend_loc=None, groups=integrated.obs['batch'].cat.categories[i], title=integrated.obs['batch'].cat.categories[i], size=20, ax=axes[i])
    sns.despine(ax=axes[i])


In [ ]:
sc.tl.rank_genes_groups(integrated, 'batch', method='wilcoxon', corr_method='benjamini-hochberg', use_raw=True, pts=True, key_added='DEG_bw_batches_wilcoxon') # key_added=''

sc.pl.rank_genes_groups(integrated, n_genes=15, sharey=False, key='DEG_bw_batches_wilcoxon')

ax_dict = sc.pl.rank_genes_groups_heatmap(integrated, n_genes=15, groupby='batch', key='DEG_bw_batches_wilcoxon', groups=['AK1', 'iPSC', 'NPC'], show_gene_labels=True, min_logfoldchange=1, dendrogram=False, cmap='viridis', use_raw=False, swap_axes=True, show=False, var_group_rotation=90)
ax_dict['heatmap_ax'].set_yticklabels(labels=ax_dict['heatmap_ax'].get_yticklabels(), fontstyle='italic')


markers = ["Pecam1", "Cdh5", "Nos3", "Acta2", "Cnn1", "Tagln", "Rgs5", "Kcnj8", "Col1a1", "Col5a1", "Dpt", "Cd19", "Ighm", "Cd14", "Cd68", "Cd3d"] # Cd3g 없음
sc.pl.stacked_violin(test3, markers, groupby='batch')

result = test3.uns['rank_genes_groups']
groups = result['names'].dtype.names
deg_wilcoxon = pd.DataFrame({group + '_' + key: result[key][group] for group in groups for key in ['names', 'logfoldchanges', 'scores', 'pvals_adj']})
deg_wilcoxon.to_csv("/data/Projects/phenomata/01.Projects/11.Vascular_Aging/03.Scanpy/20210916_scanpy_deg.csv", mode='w')


In [ ]:
# AK1 (B-lymphocyte) specific gene
genes = ['CD19', 'MS4A1'] # MS4A1 (CD20)
fig, axes = plt.subplots(1, 2, figsize=(10, 5), constrained_layout=True)
for i in range(len(genes)):
    sc.pl.umap(integrated, color=genes[i], add_outline=False, legend_loc='right margin', size=40, color_map='viridis', use_raw=True, ax=axes[i])
    axes[i].set_title(genes[i], style='italic')
    sns.despine(ax=axes[i])

# iPSC-specific gene
genes = ['POU5F1', 'SOX2', 'NANOG'] # JARID2
fig, axes = plt.subplots(1, 3, figsize=(15, 10), constrained_layout=True)
for i in range(len(genes)):
    sc.pl.umap(integrated, color=genes[i], add_outline=False, legend_loc='right margin', size=40, color_map='viridis', use_raw=True, ax=axes[i])
    axes[i].set_title(genes[i], style='italic')
    sns.despine(ax=axes[i])


# NPC-specific gene
genes = ['NCAM1', 'KMT2D']
fig, axes = plt.subplots(1, 2, figsize=(10, 5), constrained_layout=True)
for i in range(len(genes)):
    sc.pl.umap(integrated, color=genes[i], add_outline=False, legend_loc='right margin', size=40, color_map='viridis', use_raw=True, ax=axes[i])
    axes[i].set_title(genes[i], style='italic')
    sns.despine(ax=axes[i])

In [ ]:

sc.tl.leiden(test3, resolution=0.5, key_added='leiden_r05') #### 0 ~ 13 ==> 2021-09-28
sc.tl.leiden(test3, resolution=1.0, key_added='leiden_r10')
sc.pl.umap(test3, color=['batch', 'leiden_r05', 'leiden_r10'], add_outline=False, legend_loc='right margin', size=20)

fig, axes = plt.subplots(1,3)
sc.pl.umap(test3, color=['batch'], add_outline=False, legend_loc='right margin', size=20, groups=['m01'], title='1 month', ax=axes[0])
sc.pl.umap(test3, color=['batch'], add_outline=False, legend_loc='right margin', size=20, groups=['m10'], title='10 months', ax=axes[1])
sc.pl.umap(test3, color=['batch'], add_outline=False, legend_loc='right margin', size=20, groups=['m20'], title='20 months', ax=axes[2])


In [ ]:
# Diffusion pseudotime
sc.tl.diffmap(test3_endo)
sc.pl.diffmap(test3_endo, color=['batch', 'Pecam1', 'Cdh5'], add_outline=False, legend_loc='right margin', size=70, color_map='CMRmap')

sc.tl.draw_graph(test3, layout='fa', init_pos=None, neighbors_key=None) ## init_pos가 .obsm에 있는 pca, umap, paga 등이 될 수 있다.
sc.pl.draw_graph(test3, color=['batch', 'PECAM1', 'CDH5', 'phase'], add_outline=True, legend_loc='right margin', size=10, color_map='CMRmap')

start_cell = np.isin(test3_endo.obs['endo_leiden_r05'], '0') # boolean numpy array ==> array([False, False, False, ..., False, False, False])
#max_start_id = np.argmin(test3_endo.obsm['X_diffmap'][start_cell,1]) # 262
max_start_id = np.argmax(test3_endo.obsm['X_diffmap'][start_cell,1])
root_id = np.arange(len(start_cell))[start_cell][max_start_id] # 341
test3_endo.uns['iroot'] = root_id

sc.tl.dpt(test3_endo, n_branchings=1, n_dcs=10) # n_branchings를 0으로 하면 (recommended by Scanpy developer) dpt_groups가 생성 안 됨.
#computing Diffusion Pseudotime using n_dcs=10
sc.pl.dpt_groups_pseudotime(test3_endo) # 여기에서 pseudotime trajecgory 확인.

lin = ('2', '0', '3', '1') # DPT pseudotime group ordering에 맞게 배치
test3_endo.obs['dpt_groups'] = test3_endo.obs['dpt_groups'].cat.reorder_categories(list(lin), ordered=True)
sc.pl.dpt_groups_pseudotime(test3_endo) # 다시 ordering에 맞게 plotting
sc.pl.dpt_timeseries(test3_endo[:, test3_endo.var.highly_variable])


#### Diffusion pseudotime (Experimental)

In [ ]:
sc.tl.diffmap(integrated)
sc.pl.diffmap(integrated, color=['batch'], add_outline=False, legend_loc='right margin', size=70, color_map='CMRmap')


sc.tl.draw_graph(test3, layout='fa', init_pos=None, neighbors_key=None) ## init_pos가 .obsm에 있는 pca, umap, paga 등이 될 수 있다.
sc.pl.draw_graph(test3, color=['batch', 'PECAM1', 'CDH5', 'phase'], add_outline=True, legend_loc='right margin', size=10, color_map='CMRmap')

start_cell = np.isin(test3_endo.obs['endo_leiden_r05'], '0') # boolean numpy array ==> array([False, False, False, ..., False, False, False])
#max_start_id = np.argmin(test3_endo.obsm['X_diffmap'][start_cell,1]) # 262
max_start_id = np.argmax(test3_endo.obsm['X_diffmap'][start_cell,1])
root_id = np.arange(len(start_cell))[start_cell][max_start_id] # 341
test3_endo.uns['iroot'] = root_id

sc.tl.dpt(test3_endo, n_branchings=1, n_dcs=10) # n_branchings를 0으로 하면 (recommended by Scanpy developer) dpt_groups가 생성 안 됨.
#computing Diffusion Pseudotime using n_dcs=10
sc.pl.dpt_groups_pseudotime(test3_endo) # 여기에서 pseudotime trajecgory 확인.

lin = ('2', '0', '3', '1') # DPT pseudotime group ordering에 맞게 배치
test3_endo.obs['dpt_groups'] = test3_endo.obs['dpt_groups'].cat.reorder_categories(list(lin), ordered=True)
sc.pl.dpt_groups_pseudotime(test3_endo) # 다시 ordering에 맞게 plotting
sc.pl.dpt_timeseries(test3_endo[:, test3_endo.var.highly_variable])



#### Force-Directged Graph (Experimental)

In [ ]:
sc.tl.draw_graph(integrated, layout='fa', init_pos=None, neighbors_key=None) ## init_pos가 .obsm에 있는 pca, umap, paga 등이 될 수 있다.
sc.pl.draw_graph(integrated, color=['batch'], add_outline=True, legend_loc='on data', size=40, color_map='CMRmap')
